# Test Stree with AdaBoost and Bagging with different configurations

# Setup
Uncomment the next cell if STree is not already installed

In [1]:
#
# Google Colab setup
#
#!pip install git+https://github.com/doctorado-ml/stree

In [2]:
import time
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split
from stree import Stree

In [3]:
import os
if not os.path.isfile('data/creditcard.csv'):
    !wget --no-check-certificate --content-disposition http://nube.jccm.es/index.php/s/Zs7SYtZQJ3RQ2H2/download
    !tar xzf creditcard.tgz

In [4]:
random_state=1

def load_creditcard(n_examples=0):
    import pandas as pd
    import numpy as np
    import random
    df = pd.read_csv('data/creditcard.csv')
    print("Fraud: {0:.3f}% {1}".format(df.Class[df.Class == 1].count()*100/df.shape[0], df.Class[df.Class == 1].count()))
    print("Valid: {0:.3f}% {1}".format(df.Class[df.Class == 0].count()*100/df.shape[0], df.Class[df.Class == 0].count()))
    y = df.Class
    X = df.drop(['Class', 'Time', 'Amount'], axis=1).values
    if n_examples > 0:
        # Take first n_examples samples
        X = X[:n_examples, :]
        y = y[:n_examples, :]
    else:
        # Take all the positive samples with a number of random negatives
        if n_examples < 0:
            Xt = X[(y == 1).ravel()]
            yt = y[(y == 1).ravel()]
            indices = random.sample(range(X.shape[0]), -1 * n_examples)
            X = np.append(Xt, X[indices], axis=0)
            y = np.append(yt, y[indices], axis=0)
    print("X.shape", X.shape, " y.shape", y.shape)
    print("Fraud: {0:.3f}% {1}".format(len(y[y == 1])*100/X.shape[0], len(y[y == 1])))
    print("Valid: {0:.3f}% {1}".format(len(y[y == 0]) * 100 / X.shape[0], len(y[y == 0])))
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=random_state, stratify=y)
    return Xtrain, Xtest, ytrain, ytest

# data = load_creditcard(-1000) # Take all true samples + 1000 of the others
# data = load_creditcard(5000)  # Take the first 5000 samples
# data = load_creditcard(0) # Take all the samples
data = load_creditcard(-100000)

Xtrain = data[0]
Xtest = data[1]
ytrain = data[2]
ytest = data[3]

Fraud: 0.173% 492
Valid: 99.827% 284315
X.shape (100492, 28)  y.shape (100492,)
Fraud: 0.644% 647
Valid: 99.356% 99845


# Tests

## STree alone with 100.000 samples and linear kernel

In [5]:
now = time.time()
clf = Stree(max_depth=3, random_state=random_state)
clf.fit(Xtrain, ytrain)
print("Score Train: ", clf.score(Xtrain, ytrain))
print("Score Test: ", clf.score(Xtest, ytest))
print(f"Took {time.time() - now:.2f} seconds")

Score Train:  0.9985784146480154
Score Test:  0.9981093273185617
Took 73.27 seconds


## Adaboost

In [6]:
n_estimators = 10
C = 7
max_depth = 3

In [7]:
for kernel in ['linear', 'rbf', 'poly']:
    now = time.time()
    clf = AdaBoostClassifier(base_estimator=Stree(C=C, kernel=kernel, max_depth=max_depth, random_state=random_state), algorithm="SAMME", n_estimators=n_estimators, random_state=random_state)
    clf.fit(Xtrain, ytrain)
    score_train = clf.score(Xtrain, ytrain)
    score_test = clf.score(Xtest, ytest)
    print(f"Kernel: {kernel}\tTime: {time.time() - now:.2f} seconds\tScore Train: {score_train:.7f}\tScore Test: {score_test:.7f}")

Kernel: linear	Time: 93.78 seconds	Score Train: 0.9983083	Score Test: 0.9983083
Kernel: rbf	Time: 18.32 seconds	Score Train: 0.9935602	Score Test: 0.9935651
Kernel: poly	Time: 69.68 seconds	Score Train: 0.9973132	Score Test: 0.9972801


## Bagging

In [8]:
n_estimators = 10
C = 7
max_depth = 3

In [9]:
for kernel in ['linear', 'rbf', 'poly']:
    now = time.time()
    clf = BaggingClassifier(base_estimator=Stree(C=C, kernel=kernel, max_depth=max_depth, random_state=random_state), n_estimators=n_estimators, random_state=random_state)
    clf.fit(Xtrain, ytrain)
    score_train = clf.score(Xtrain, ytrain)
    score_test = clf.score(Xtest, ytest)
    print(f"Kernel: {kernel}\tTime: {time.time() - now:.2f} seconds\tScore Train: {score_train:.7f}\tScore Test: {score_test:.7f}")

Kernel: linear	Time: 387.06 seconds	Score Train: 0.9985784	Score Test: 0.9981093
Kernel: rbf	Time: 144.00 seconds	Score Train: 0.9992750	Score Test: 0.9983415
Kernel: poly	Time: 101.78 seconds	Score Train: 0.9992466	Score Test: 0.9981757
